In [1]:
import graphlab

## Read product review data

In [2]:
product = graphlab.SFrame('amazon_baby.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\zby0902\AppData\Local\Temp\graphlab_server_1529293849.log.0


This non-commercial license of GraphLab Create for academic use is assigned to bzha0010@student.monsh.edu and will expire on June 12, 2019.


## EDA

In [3]:
product.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build word count vector

In [4]:
product['word_count'] = graphlab.text_analytics.count_words(product['review'])

In [5]:
product.head(5)

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5L, 'stink': 1L,'because': 1L, 'order ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 2L, 'highly': 1L, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1L, 'and':3L, 'love': 2L, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'parents!!':1L, 'all': 2L, 'puppe ..."


In [6]:
product['name'].show()

Canvas is accessible via web browser at the URL: http://localhost:55475/index.html
Opening Canvas in default web browser.


## Explore the most popular product

In [7]:
vuli = product[product['name'] == 'Vulli Sophie the Giraffe Teether']

In [8]:
len(vuli)

785

In [9]:
graphlab.canvas.set_target('headless')

In [10]:
vuli['rating'].show(view='Categorical')

Canvas is accessible via web browser at the URL: http://localhost:55475/index.html
Opening Canvas in default web browser.


## Build a simple sentiment classifier

### Define the positive and negative

In [11]:
#ignore all rating 3
product= product[product['rating'] != 3]

In [12]:
#positive means 4 or 5
#negative means 1 or 2
product['sentiment'] = product['rating'] >= 4

## Train the classifier

In [13]:
train, test = product.random_split(0.8,seed=0)

In [17]:
sentiment_model = graphlab.logistic_classifier.create(train,target='sentiment',features=['word_count'],validation_set=test)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

...

| 6         | 13       | 3.000000  | 3.312000     | 0.899991          | 0.825967            |

| 10        | 18       | 1.000000  | 4.972487     | 0.988715          | 0.916256            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

## Evaluate the model

In [21]:
sentiment_model.evaluate(test,metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+-------+------+
 | threshold |      fpr       |      tpr       |   p   |  n   |
 +-----------+----------------+----------------+-------+------+
 |    0.0    |      1.0       |      1.0       | 27976 | 5328 |
 |   1e-05   | 0.909346846847 | 0.998856162425 | 27976 | 5328 |
 |   2e-05   | 0.896021021021 | 0.998748927652 | 27976 | 5328 |
 |   3e-05   | 0.886448948949 | 0.998462968259 | 27976 | 5328 |
 |   4e-05   | 0.879692192192 | 0.998284243637 | 27976 | 5328 |
 |   5e-05   | 0.875187687688 | 0.998212753789 | 27976 | 5328 |
 |   6e-05   | 0.872184684685 | 0.998177008865 | 27976 | 5328 |
 |   7e-05   | 0.868618618619 | 0.998034029168 | 27976 | 5328 |
 |   8e-05   | 0.864677177177 | 0.997998284244 | 27976 | 5328 |
 |   9e-05   | 0.860735735736 | 0.997962539319 | 27976 | 5328 |
 +-----------+----------------+----------------+-------+------

In [22]:
sentiment_model.show(view='Evaluation')

Canvas is accessible via web browser at the URL: http://localhost:55475/index.html
Opening Canvas in default web browser.


## Apply the learned model to understand the sentiment for the vulli toy

In [23]:
vuli['predicted_sentiment'] = sentiment_model.predict(vuli,output_type='probability')

In [24]:
vuli.head(5)

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'and': 1L, 'all': 1L,'because': 1L, 'it': 1L, ...",0.999513023521
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'and': 1L, 'right': 1L,'help': 1L, 'just': 1L, ...",0.999320678306
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'and': 2L, 'all': 1L,'latex.': 1L, 'being': ...",0.013558811687
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'and': 2L, 'one!': 1L,'all': 1L, 'love': 1L, ...",0.995769474148
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'and': 3L, 'all': 1L,'old': 1L, 'her.': 1L, ...",0.662374415673


## Sort the review based on prediceted sentiment

In [30]:
vuli = vuli.sort('predicted_sentiment',0)

In [31]:
vuli.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'giggles': 1L, 'all':1L, ""violet's"": 2L, ...",1.0
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'peace': 1L, 'month':1L, 'bright': 1L, ...",0.999999999703
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'all': 2L, 'pops': 1L,'existence.': 1L, ...",0.999999999392
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'all': 2L, ""don't"": 1L,'(literally).so': 1L, ...",0.99999999919
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'cute': 1L, 'all': 1L,'reviews.': 2L, 'just': ...",0.999999998657
Vulli Sophie the GiraffeTeether ...,Sophie the Giraffe is theperfect teething toy. ...,5.0,"{'just': 2L, 'both': 1L,'month': 1L, 'ears,': ...",0.999999997108
Vulli Sophie the GiraffeTeether ...,Sophie la giraffe isabsolutely the best toy ...,5.0,"{'and': 5L, 'the': 1L,'all': 1L, 'old': 1L, ...",0.999999995589
Vulli Sophie the GiraffeTeether ...,My 5-mos old son took tothis immediately. The ...,5.0,"{'just': 1L, 'shape': 2L,'mutt': 1L, '""dog': 1L, ...",0.999999995573
Vulli Sophie the GiraffeTeether ...,My nephews and my fourkids all had Sophie in ...,5.0,"{'and': 4L, 'chew': 1L,'all': 1L, 'perfect;': ...",0.999999989527
Vulli Sophie the GiraffeTeether ...,Never thought I'd see myson French kissing a ...,5.0,"{'giggles': 1L, 'all':1L, 'out,': 1L, 'over': ...",0.999999985069


## Explore some of the most positive and negative reviews

In [32]:
vuli[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

In [40]:
vuli[['review','rating']][-1]

{'rating': 1.0,
 'review': "My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"}

## Quiz question

In [41]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [50]:
def get_selected_count(x,word):
    selected_count = {}
    if word in x:
        return x[word]
    else:
        return 0

In [54]:
for word in selected_worbds:
    product['{}'.format(word)] =  product['word_count'].apply(lambda x: get_selected_count(x,word))

In [71]:
for word in selected_words:
    print(product['{}'.format(word)].sum(),word)

(2002L, 'awesome')
(42420L, 'great')
(873L, 'fantastic')
(1305L, 'amazing')
(40277L, 'love')
(659L, 'horrible')
(3197L, 'bad')
(673L, 'terrible')
(345L, 'awful')
(131L, 'wow')
(1057L, 'hate')


## Train a new classifier based on the selected words

In [73]:
train_data,test_data = product.random_split(.8, seed=0)

In [78]:
selected_words_model = graphlab.logistic_classifier.create(train_data,target='sentiment',features=selected_words,validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.317144     | 0.844269          | 0.843142            |

| 5         | 6        | 0.384961     | 0.844269          | 0.843142            |

| 6         | 7        | 0.451781     | 0.844269          | 0.843142            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [80]:
selected_words_model['coefficients'].sort('value',-1)

name,index,class,value,stderr
terrible,None,1,-2.09049998487,0.0967241912229
horrible,None,1,-1.99651800559,0.0973584169028
awful,None,1,-1.76469955631,0.134679803365
hate,None,1,-1.40916406276,0.0771983993506
bad,None,1,-0.985827369929,0.0433603009142
wow,None,1,-0.0541450123333,0.275616449416
great,None,1,0.883937894898,0.0217379527921
fantastic,None,1,0.891303090304,0.154532343591
amazing,None,1,0.892802422508,0.127989503231
awesome,None,1,1.05800888878,0.110865296265


## Compare the new model and the old model

In [81]:
selected_words_model.evaluate(test_data,metric='accuracy')

{'accuracy': 0.8431419649291376}

In [86]:
sentiment_model.evaluate(test,metric='accuracy')

{'accuracy': 0.916256305548883}

In [95]:
product['predicted_sentiment'] = sentiment_model.predict(product,output_type='probability')

In [98]:
data = product[product['name'] == 'Baby Trend Diaper Champ'].sort('predicted_sentiment',0)[0]

In [99]:
selected_words_model.predict(data, output_type='probability')

dtype: float
Rows: 1
[0.796940851290673]

In [101]:
data

{'amazing': 0L,
 'awesome': 0L,
 'awful': 0L,
 'bad': 0L,
 'fantastic': 0L,
 'great': 0L,
 'hate': 0L,
 'horrible': 0L,
 'love': 0L,
 'name': 'Baby Trend Diaper Champ',
 'predicted_sentiment': 0.9999999372669541,
 'rating': 5.0,
 'review': 'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffini

## See if our model can beat the majority class classifier

In [107]:
float(test_data[test_data['rating']>=4].num_rows())/test_data.num_rows()

0.8400192169108815